In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict, load_dataset
import torch
import random

c:\Users\WONJONGHYEON\anaconda3\envs\CP2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 데이터 불러오기 및 분리

In [738]:
df=pd.read_csv('train.csv').drop(['ID'], axis=1)
trans_aug=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\cp2-competition\augment_csv\trans_aug.csv')



In [739]:
df

,문장,유형,극성,시제,확실성,label
0,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...
16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


In [740]:
trans_aug

,문장,유형,극성,시제,확실성,label
0,"비가 오는 지역에는 폭풍과 함께 폭풍이 있을 것이며, 일부 지역에는 폭풍이 떨어지는...",예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
1,"""당신이 처음부터 OTT 사업을 성장시키고 싶었다면, Cupplay의 'Time t...",예측형,부정,과거,확실,예측형-부정-과거-확실
2,바다에는 폭풍이 폭풍으로 타는 곳들이 있을 것이다.,예측형,미정,미래,불확실,예측형-미정-미래-불확실
3,정부는 단기 유동성에 기여하는 무역 자금을 지원하고 금융으로 기업 자금을 적극적으로...,예측형,긍정,미래,확실,예측형-긍정-미래-확실
4,외국 석탄 발전소와 광산은 또한 $ 1.9 억을 처분 할 계획이다.,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
...,...,...,...,...,...,...
323,시베리아 전염병과 마찬가지로 우리가 이미 알고있는 전염병이나 새로운 감염은 동물 전...,대화형,미정,미래,불확실,대화형-미정-미래-불확실
324,새로운 ESL Pro Tour Starcraft 2는 기존의 Starcraft 2 ...,사실형,미정,미래,확실,사실형-미정-미래-확실
325,거기 거리가 있나요?,대화형,미정,현재,불확실,대화형-미정-현재-불확실
326,KF-X의 운영 대상인 공군은 10월 1일부터 10월 30일까지 한국 유형 전투기 ...,사실형,미정,미래,확실,사실형-미정-미래-확실


In [545]:
label_table = df['label'].value_counts().to_frame()
label_table.head()

,label
사실형-긍정-과거-확실,7113
사실형-긍정-현재-확실,4743
추론형-긍정-현재-확실,1101
사실형-긍정-미래-확실,649
추론형-긍정-과거-확실,335


In [546]:
aug_label_table = trans_aug['label'].value_counts().to_frame()
aug_label_table.head()

,label
예측형-긍정-미래-불확실,263
예측형-긍정-미래-확실,107
대화형-미정-현재-불확실,47
사실형-미정-미래-확실,42
사실형-미정-미래-불확실,38


In [547]:
labels = label_table[label_table.label>=10].index
aug_labels = aug_label_table[aug_label_table.label>=10].index

In [550]:
idx_lst, aug_idx_lst = [], []

for idx, label in enumerate(df['label']):
    if not label in list(labels):
        idx_lst.append(idx)

for idx, label in enumerate(trans_aug['label']):
    if not label in list(aug_labels):
        aug_idx_lst.append(idx)

In [552]:
df_in=df.drop(idx_lst)
df_in.reset_index(drop=True, inplace=True)

df_out=df.iloc[idx_lst]
df_out.reset_index(drop=True, inplace=True)
df_out=df_out.sample(frac=1)


In [553]:
aug_in=trans_aug.drop(aug_idx_lst)
aug_in.reset_index(drop=True, inplace=True)

aug_out=trans_aug.iloc[aug_idx_lst]
aug_out.reset_index(drop=True, inplace=True)
aug_out=aug_out.sample(frac=1)

In [570]:
from sklearn.model_selection import train_test_split
df_train, df_val, y_t, y_v = train_test_split(df_in[['문장', '유형', '극성', '시제', '확실성']], df_in[['label']], test_size=0.2, stratify=df_in['label'], random_state=1)
aug_train, aug_val, aug_y_t, aug_y_v = train_test_split(aug_in[['문장', '유형', '극성', '시제', '확실성']], aug_in[['label']], test_size=0.2, stratify=aug_in['label'], random_state=1)

In [573]:
aug_y_t.value_counts(normalize=True)

label        
예측형-긍정-미래-불확실    0.375000
예측형-긍정-미래-확실     0.151786
대화형-미정-현재-불확실    0.067857
사실형-미정-미래-확실     0.060714
대화형-미정-미래-불확실    0.053571
사실형-미정-미래-불확실    0.053571
추론형-미정-미래-불확실    0.046429
사실형-미정-현재-확실     0.037500
예측형-긍정-현재-확실     0.032143
예측형-미정-미래-불확실    0.030357
예측형-긍정-과거-확실     0.025000
예측형-긍정-현재-불확실    0.023214
대화형-미정-과거-불확실    0.021429
예측형-긍정-과거-불확실    0.021429
dtype: float64

In [572]:
aug_y_v.value_counts(normalize=True)

label        
예측형-긍정-미래-불확실    0.375887
예측형-긍정-미래-확실     0.156028
대화형-미정-현재-불확실    0.063830
사실형-미정-미래-불확실    0.056738
사실형-미정-미래-확실     0.056738
대화형-미정-미래-불확실    0.049645
추론형-미정-미래-불확실    0.049645
사실형-미정-현재-확실     0.035461
예측형-긍정-현재-확실     0.035461
예측형-긍정-과거-확실     0.028369
예측형-미정-미래-불확실    0.028369
대화형-미정-과거-불확실    0.021277
예측형-긍정-과거-불확실    0.021277
예측형-긍정-현재-불확실    0.021277
dtype: float64

In [555]:
len(df_out), len(aug_out)

(92, 52)

In [556]:
df_train = pd.concat([df_train, df_out[:41].drop(['label'], axis=1)], axis=0)
df_val = pd.concat([df_val, df_out[41:].drop(['label'], axis=1)], axis=0)
aug_train = pd.concat([aug_train, aug_out[:26].drop(['label'], axis=1)], axis=0)
aug_val = pd.concat([aug_val, aug_out[26:].drop(['label'], axis=1)], axis=0)

In [563]:
df_train = pd.concat([df_train, aug_train], axis=0)
df_train.drop_duplicates(inplace=True)
df_val = pd.concat([df_val, aug_val], axis=0)
df_val.drop_duplicates(inplace=True)

In [808]:
df_train['유형'].value_counts(), df_val['유형'].value_counts()

(사실형    10851
 추론형     1722
 대화형      480
 예측형      372
 Name: 유형, dtype: int64,
 사실형    2732
 추론형     445
 대화형     131
 예측형     112
 Name: 유형, dtype: int64)

In [746]:
df_train['극성'].value_counts(), df_val['극성'].value_counts()

(긍정    12764
 부정      440
 미정      221
 Name: 극성, dtype: int64,
 긍정    3210
 부정     130
 미정      80
 Name: 극성, dtype: int64)

In [744]:
df_train['시제'].value_counts(), df_val['시제'].value_counts()

(과거    6421
 현재    5495
 미래    1509
 Name: 시제, dtype: int64,
 과거    1624
 현재    1413
 미래     383
 Name: 시제, dtype: int64)

In [743]:
df_train['확실성'].value_counts(), df_val['확실성'].value_counts()

(확실     12199
 불확실     1226
 Name: 확실성, dtype: int64,
 확실     3072
 불확실     348
 Name: 확실성, dtype: int64)

In [741]:
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')

In [742]:
df_train

,문장,유형,극성,시제,확실성
0,오는 1월 말에 개최될 예정이던 아시아 최대 미술품 거래 장터인 아트바젤홍콩이 코로...,사실형,긍정,과거,확실
1,어우야담에 따르면 당시 사람들은 그를 보고 어람좌객이라고 불렀다,사실형,긍정,과거,확실
2,봉준호 감독은 예상치 못했다며 고개를 내저었고 시상식장을 가득 메운 이들의 우렁찬 ...,사실형,긍정,과거,확실
3,어제에 이어 프라이버시 대표 코인인 지코인a과 지캐시a가 각각 1 1로 상승 a 1...,대화형,긍정,과거,확실
4,나는 내 나름의 등반 방식에 대해 포기 알피니즘이라는 말을 붙였습니다,대화형,긍정,과거,확실
...,...,...,...,...,...
13420,리그에 참여하지 않는 사용자가 함께 즐길 수있는 구조를 만들기 위해 a 측면을 강화...,사실형,미정,현재,확실
13421,뉴욕 타임즈는 이번 주 회의는 1년 이후 가장 큰 금리 인프라를 논의하는 것으로 보...,예측형,긍정,과거,확실
13422,1일 기상청에 따르면 1일 오후 늦게부터 경기의 밤과 남쪽과 서쪽 광장 사이에 남쪽...,예측형,미정,미래,확실
13423,영웅은 1주 연속으로 배제되지 않으며 팬과 플레이어의 의견을 적극적으로 감소함으로써...,추론형,미정,미래,확실


## rs, rd

In [908]:
import spacy
from spacy.lang.ko.examples import sentences 
nlp = spacy.load("ko_core_news_sm") #python -m spacy download ko_core_news_sm

### 불용어 가져오기

In [909]:
# open 불용어.txt, stop_words_set(불용어 사전) 만듬.
import locale

# 지역 설정 변경
locale.setlocale(locale.LC_COLLATE, 'ko_KR.UTF-8')

stop_words = {}

with open('불용어.txt', 'r') as f:
    for line in f:
        line = line.strip('\n')
        line = line.split(':')  # 줄 바꿈 문자 제거
        key = line[0] #key
        value = line[1].split(', ')  # value
        value.sort(key=locale.strxfrm)
        stop_words[key] = value  # 딕셔너리에 추가


# stop_words_set
stop_words_lst=[]
for _, value in stop_words.items():
    stop_words_lst.extend(value)

stop_words_set=set(stop_words_lst)
stop_words_set.update(nlp.Defaults.stop_words)

In [910]:
neg_adv_lst=['못', '안', '전혀', '결코', '별로', '그리', '도대체', '도저히', '절대', '그다지', '절대로', '도무지', '채', '미처', '좀처럼', '여간', '영', '비단', '차마', '통', '구태여']

In [911]:
for neg_adv in neg_adv_lst:
    if neg_adv in stop_words_set:
        stop_words_set.remove(neg_adv)

In [912]:
'수' in stop_words_set

True

In [913]:
stop_words_set.remove('수') #예측에 많이 쓰이는 의존명사

### random deletion

In [973]:
def random_deletion(words, p):

    new_words=[]
    for token in words:
        r=random.uniform(0, 1)
        spt = token.lemma_.split('+')
        if r>p:
            if token.text in stop_words_set: #1-p의 확률로 불용어를 제거한다.
                continue
            if (token.pos_ =='ADV') and (token.dep_ not in ['ROOT', 'fixed', 'conj']) and (token.text not in neg_adv_lst): #1-p의 확률로 부정 부사를 제외한 부사 제거
                continue
            if token.pos_ in ['ADJ', 'NUM'] and token.dep_ not in ['ROOT', 'fixed', 'conj']: #1-p확률로 수식하는 단어를 제거
                continue
            if token.pos_ != 'VERB' and len(spt) == 2 and spt[0] != '나' and spt[-1] in ['은', '는', '이', '가', '을', '를', '에', '에게', '의', '께서']:
                new_words.append(''.join(spt[:-1]))
                continue
        
        new_words.append(token.text) #포함되는 토큰을 담는다.

    if len(new_words) < 4:
      new_words = [token.text for token in words]
      
    return new_words

In [915]:
doc = nlp("하늘을 나는 꿈을 꾸는 꼬마 자동차")
for token in doc:
    
    spt = token.lemma_.split('+')
    if token.pos_ != 'VERB' and len(spt) == 2 and spt[0] != '나' and spt[-1] in ['은', '는', '이', '가', '을', '를', '에', '에게', '의', '께서', '께']:
        print(''.join(spt[:-1]))
    else:
        print(token.text)



하늘
나는
꿈
꾸는
꼬마
자동차


In [ ]:
doc = nlp(df_train.iloc[[98]]['문장'][98])
for token in doc:
    print(token.text, token.pos_, token.dep_, token.lemma_)

In [972]:
df_train.iloc[[98]]

,문장,유형,극성,시제,확실성
98,1조원 영업이익을 내는 기업도 기업어음 발행에 차질이 빚어져 1월 위기설이 떠돌았던...,대화형,긍정,현재,확실


In [ ]:
random_deletion(doc, 0.5)


### random swap

In [974]:
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	if len(new_words) > 4:
		random_idx_1 = random.randint(1, len(new_words)-2) #양 끝 단어는 교체하지 못하게 함.
		random_idx_2 = random_idx_1
		counter = 0

		while random_idx_2 == random_idx_1:
			random_idx_2 = random.randint(1, len(new_words)-2)
			counter += 1
			if counter > 3:
				return new_words

		new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words


### EDA(Easy Data Augmentation)

In [975]:
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')

In [976]:
def EDA(sentence, alpha_rs=0.15, p_rd=0.5): 

	docs = nlp(sentence)
	words = random_deletion(docs, p_rd)
	
	if words:
		num_words=len(words)
		if num_words<20:
			n_rs = max(1, int(alpha_rs*num_words))
			a_words = random_swap(words, n_rs)
			augmented_sentences = " ".join(a_words)

		else:	augmented_sentences = " ".join(words)

	return augmented_sentences

In [977]:
def df_augmentation(df):

    df_aug = df.copy()

    for i in range(len(df)):
        
        r = random.uniform(0, 1)
        sentence_type = df['유형'][i]
        sentence_polarity = df['극성'][i]
        sentence_certainty = df['확실성'][i]
        sentence_tense = df['시제'][i]


        if sentence_type == '추론형':
            if r <= 0.5:
                df_aug['문장'][i] = EDA(df_aug['문장'][i])
                continue

        if sentence_type != '사실형' and sentence_type != '추론형':
            df_aug['문장'][i] = EDA(df_aug['문장'][i])
            continue

        
        if sentence_certainty == '불확실':
            df_aug['문장'][i] = EDA(df_aug['문장'][i])
            continue

        if sentence_polarity != '긍정':
            df_aug['문장'][i] = EDA(df_aug['문장'][i])
            continue

        if sentence_tense == '미래':
            if r <= 0.7:
                df_aug['문장'][i] = EDA(df_aug['문장'][i])
                continue

        else:
            if r<=0.3:
                df_aug['문장'][i] = EDA(df_aug['문장'][i])

    return df_aug



In [ ]:
doc = nlp(df_train.iloc[[98]]['문장'][98])
for token in doc:
    print(token.text, token.pos_, token.dep_, token.lemma_)

In [979]:
df_train_aug_1 = df_augmentation(df_train)
df_train_aug_2 = df_augmentation(df_train)
df_train_aug_3 = df_augmentation(df_train)
df_train_aug = pd.concat([df_train_aug_1, df_train_aug_2, df_train_aug_3, df_train], axis=0).drop_duplicates().reset_index(drop=True)

In [986]:
df_val_aug_1 = df_augmentation(df_val)
df_val_aug_2 = df_augmentation(df_val)
df_val_aug_3 = df_augmentation(df_val)
df_val_aug = pd.concat([df_val_aug_1, df_val_aug_2, df_val_aug_3, df_val], axis=0).drop_duplicates().reset_index(drop=True)

In [1003]:
len(df_train)

13425

In [1004]:
len(df_train_aug)

30708

#### value_counts

In [999]:
df_train.유형.value_counts(), df_train_aug['유형'].value_counts()

(사실형    10851
 추론형     1722
 대화형      480
 예측형      372
 Name: 유형, dtype: int64,
 사실형    22650
 추론형     4854
 대화형     1727
 예측형     1477
 Name: 유형, dtype: int64)

In [993]:
df_train.극성.value_counts(), df_train_aug['극성'].value_counts()

(긍정    12764
 부정      440
 미정      221
 Name: 극성, dtype: int64,
 긍정    26207
 부정     1277
 미정      617
 Name: 극성, dtype: int64)

In [995]:
df_train.시제.value_counts(), df_train_aug['시제'].value_counts()

(과거    6421
 현재    5495
 미래    1509
 Name: 시제, dtype: int64,
 과거    12559
 현재    11225
 미래     4317
 Name: 시제, dtype: int64)

In [996]:
df_train.확실성.value_counts(), df_train_aug['확실성'].value_counts()

(확실     12199
 불확실     1226
 Name: 확실성, dtype: int64,
 확실     24527
 불확실     3574
 Name: 확실성, dtype: int64)

In [1005]:
df_val_aug.유형.value_counts(), df_val.유형.value_counts()

(사실형    5742
 추론형    1274
 대화형     483
 예측형     447
 Name: 유형, dtype: int64,
 사실형    2732
 추론형     445
 대화형     131
 예측형     112
 Name: 유형, dtype: int64)

In [1006]:
df_val_aug.극성.value_counts(), df_val.극성.value_counts()

(긍정    7132
 부정     508
 미정     306
 Name: 극성, dtype: int64,
 긍정    3210
 부정     130
 미정      80
 Name: 극성, dtype: int64)

In [1007]:
df_val_aug.시제.value_counts(), df_val.시제.value_counts()

(과거    3422
 현재    3166
 미래    1358
 Name: 시제, dtype: int64,
 과거    1624
 현재    1413
 미래     383
 Name: 시제, dtype: int64)

In [1008]:
df_val_aug.확실성.value_counts(), df_val.확실성.value_counts()

(확실     6585
 불확실    1361
 Name: 확실성, dtype: int64,
 확실     3072
 불확실     348
 Name: 확실성, dtype: int64)

In [1023]:
df_train_aug.isna().sum()

문장     0
유형     0
극성     0
시제     0
확실성    0
dtype: int64

In [1024]:
df_val_aug.isna().sum()

문장     0
유형     0
극성     0
시제     0
확실성    0
dtype: int64

In [1025]:
df_train_aug.to_csv('df_train_aug.csv', index=False, encoding='utf-8-sig')

In [1026]:
df_val.to_csv('df_val_aug.csv', index=False, encoding='utf-8-sig')

In [400]:
df_under_type = df[df['유형']==1].sample(2000)
df_under_polar = df[df['극성']==0].sample(2000)
df_under_certainty = df[df['확실성']==1].sample(2000)
df_under_tense = df[df['시제']!=1].sample(2000)

In [402]:
df_under = pd.concat([df_under_type, df_under_polar, df_under_certainty, df_under_tense]).drop_duplicates()

In [399]:
df_aug_=pd.concat([df_aug_, df_type, df_polar, df_certainty, df_tense]).drop_duplicates()


,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 치고 곳이 지역에선 우박이 떨어지는 일부 있을 예정이다,2,0,1,0
1,비가 떨어지는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 내리는 ...,2,0,1,0
2,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 우박이 있을 곳이 떨어지는...,2,0,1,0
3,비가 곳이 지역에서는 천둥번개가 치고 일부 우박이 떨어지는 내리는 예정이다,2,0,1,0
4,비가 내리는 곳이 돌풍과 치고 일부 지역에선 우박이 떨어지는 지역에서는 예정이다,2,0,1,0
...,...,...,...,...,...
16286,산재로 인해 노동자가 사망할 경우 도급인에게는 1년 이하 징역 또는 1억원 이하 벌...,1,0,1,1
16311,장중 대규모 주식을 매도하면 주가가 급락할 수 있어 장이 끝난 뒤 매매에 나선다,1,0,1,1
16340,중국 터키 스웨덴 등은 조만간 시범운영에 나설 예정이다,1,0,1,1
16470,채권 만기는 내년 1월로 이를 상환하지 못하면 디폴트 위기를 맞게 된다,1,0,1,1


In [403]:
pd.concat([df_aug_, df_under]).drop_duplicates()

,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 치고 곳이 지역에선 우박이 떨어지는 일부 있을 예정이다,2,0,1,0
1,비가 떨어지는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 내리는 ...,2,0,1,0
2,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 우박이 있을 곳이 떨어지는...,2,0,1,0
3,비가 곳이 지역에서는 천둥번개가 치고 일부 우박이 떨어지는 내리는 예정이다,2,0,1,0
4,비가 내리는 곳이 돌풍과 치고 일부 지역에선 우박이 떨어지는 지역에서는 예정이다,2,0,1,0
...,...,...,...,...,...
14005,또한 a씨가 자신이 근무한 편의점에서 1일 동안 최소 1만원 이상의 돈을 들여 상품...,1,0,0,0
3872,오유리 빗썸경제연구소 정책연구팀장은 국내 규제가 미국의 동향을 따를 가능성이 높은 ...,1,0,0,1
14875,보라네트워크는 카카오게임즈의 계열사다,1,0,2,1
5136,컴투스는 최근 크리티카 글로벌의 a1a 블록체인 생태계 합류를 위해 참여자들이 직접...,1,0,0,1


In [404]:
df_aug = pd.concat([df_aug_, df_under]).drop_duplicates()

In [406]:
len(df_aug)

28124

In [409]:
df_aug.확실성.value_counts()

1    20608
0     7516
Name: 확실성, dtype: int64

In [410]:
df_aug.to_csv('df_aug.csv', index=False, encoding='utf-8-sig')